# COVER

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw
from scipy.ndimage import label, find_objects
from collections import Counter

def find_color_blocks(image, margin=10):
    """
    Identifies all bounding boxes of color blocks in the image.
    Arguments:
    - image: PIL Image object
    - margin: Extra margin around the detected color block
    Returns:
    - List of bounding boxes (x_min, y_min, x_max, y_max)
    """
    img = image.convert("RGB")
    img_data = np.array(img)
    width, height = img.size

    # Define color threshold (greenish range)
    min_r, max_r, min_g, max_g, min_b, max_b = 0, 255, 100, 255, 100, 255
    mask = (
        (img_data[:, :, 0] >= min_r) & (img_data[:, :, 0] <= max_r) &
        (img_data[:, :, 1] >= min_g) & (img_data[:, :, 1] <= max_g) &
        (img_data[:, :, 2] >= min_b) & (img_data[:, :, 2] <= max_b)
    )

    # Find connected components
    labeled, num_features = label(mask)
    slices = find_objects(labeled)

    bounding_boxes = []
    for s in slices:
        x_min, x_max = s[1].start, s[1].stop
        y_min, y_max = s[0].start, s[0].stop
        bounding_boxes.append((x_min, y_min, x_max, y_max))

    return bounding_boxes

def find_closest_to_center(image, bounding_boxes):
    """
    Finds the bounding box closest to the center of the image.
    Arguments:
    - image: PIL Image object
    - bounding_boxes: List of bounding boxes (x_min, y_min, x_max, y_max)
    Returns:
    - Closest bounding box to the center
    """
    width, height = image.size
    center_x, center_y = width // 2, height // 2

    def distance_to_center(box):
        x_min, y_min, x_max, y_max = box
        box_center_x = (x_min + x_max) // 2
        box_center_y = (y_min + y_max) // 2
        return ((box_center_x - center_x) ** 2 + (box_center_y - center_y) ** 2) ** 0.5

    closest_box = min(bounding_boxes, key=distance_to_center)
    return closest_box

def analyze_common_size(bounding_boxes):
    """
    Analyze the most common size of the bounding boxes.
    Arguments:
    - bounding_boxes: List of bounding boxes (x_min, y_min, x_max, y_max)
    Returns:
    - Most common size (width, height)
    """
    sizes = [(x_max - x_min, y_max - y_min) for x_min, y_min, x_max, y_max in bounding_boxes]
    size_counts = Counter(sizes)
    most_common_size = size_counts.most_common(1)[0][0]  # Get the most common size
    return most_common_size

def adjust_to_common_size_with_fixed_right(box, common_size):
    """
    Adjust bounding box to the most common size while keeping the right edge fixed.
    Arguments:
    - box: Current bounding box (x_min, y_min, x_max, y_max)
    - common_size: Most common size (width, height)
    Returns:
    - Adjusted bounding box
    """
    x_min, y_min, x_max, y_max = box
    current_width = x_max - x_min
    current_height = y_max - y_min
    target_width, target_height = common_size

    # Adjust the left and top edges to match the common size
    new_x_min = max(0, x_max - target_width)  # Keep the right edge fixed, adjust the left edge
    new_y_min = max(0, y_min)  # Keep the top edge fixed
    new_y_max = new_y_min + target_height  # Adjust the bottom edge

    return new_x_min, new_y_min, x_max, new_y_max

def process_images_with_adjusted_closest_box(input_folder, output_folder, resize_size=(256, 256)):
    """
    Process all images, find the most common size of the closest bounding box,
    and adjust the bounding box near the center to that size.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_closest_boxes = []

    # Step 1: Collect all closest bounding boxes from images
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)
            bounding_boxes = find_color_blocks(image)
            if bounding_boxes:
                closest_box = find_closest_to_center(image, bounding_boxes)
                all_closest_boxes.append(closest_box)

    # Step 2: Determine the most common size of closest bounding boxes
    if not all_closest_boxes:
        print("No bounding boxes found.")
        return
    common_size = analyze_common_size(all_closest_boxes)
    print(f"Most common size of closest bounding boxes: {common_size}")

    # Step 3: Adjust closest bounding box to the most common size
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)

            bounding_boxes = find_color_blocks(image)
            if bounding_boxes:
                closest_box = find_closest_to_center(image, bounding_boxes)

                # Adjust closest bounding box to the most common size
                adjusted_closest_box = adjust_to_common_size_with_fixed_right(closest_box, common_size)

                # Crop and save the result
                cropped_image = image.crop(adjusted_closest_box)

                # Resize the cropped image to 256x256
                resized_image = cropped_image.resize(resize_size)

                # Save the result
                output_path = os.path.join(output_folder, filename)
                resized_image.save(output_path)
                print(f"Processed and saved: {output_path}")
            else:
                print(f"No color block found in: {filename}")

# Example usage:
input_folder = './data/samsung/temp/NH3'  # './data/samsung/temp/NO2'
output_folder = './data/samsung/samsung/NH3/cover/'  # './data/samsung/samsung/NO2/cover/'
process_images_with_adjusted_closest_box(input_folder, output_folder)

Kích thước phổ biến nhất của closest_box: (671, 664)
Processed and saved: ./samsung/NH3/cover/0iOM6_ngochanpham274@gmail.com_2024-11-21 12_52_30_NH3_5ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/cover/1iVDC_ngochanpham274@gmail.com_2024-11-21 12_04_02_NH3_0,25ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/cover/2qR82_ngochanpham274@gmail.com_2024-11-21 12_00_28_NH3_0,09ppm_001_samsung test__.jpg
Processed and saved: ./samsung/NH3/cover/3iW5i_ngochanpham274@gmail.com_2024-11-21 11_48_30_NH3_0,01ppm_003_samsung test_10.8768228_106.6782789.jpg
Processed and saved: ./samsung/NH3/cover/3IwPn_ngochanpham274@gmail.com_2024-11-21 11_59_52_NH3_0,075ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/cover/4fkPP_ngochanpham274@gmail.com_2024-11-21 12_50_09_NH3_4ppm_001_samsung test__.jpg
Processed and saved: ./samsung/NH3/cover/5RgJx_ngochanpham274@gmail.com_2024-11-21 11_55_08_NH3_0,05ppm_002_samsung test_10.8768228_106.6782789.jpg
Processed and saved: 

# MIDDLE

In [ ]:
def crop_and_resize_center_square(input_folder, output_folder, resize_size=(256, 256)):
    """
    Crop a square at the center of the image with an area equal to 1/3 of the original image's area,
    then resize it to 256x256 and save it to another folder.
    Arguments:
    - input_folder: Folder containing resized images (256x256)
    - output_folder: Folder to save the processed images
    - resize_size: Target size (width, height) for the resized image
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            input_path = os.path.join(input_folder, filename)
            image = Image.open(input_path)
            width, height = image.size

            # Determine the size of the square with an area equal to 1/3 of the original image
            side_length = int((width * height / 3) ** 0.5)

            # Calculate coordinates to center the square in the image
            center_x, center_y = width // 2, height // 2
            x_min = max(0, center_x - side_length // 2)
            y_min = max(0, center_y - side_length // 2)
            x_max = min(width, x_min + side_length)
            y_max = min(height, y_min + side_length)

            # Crop the center square
            cropped_image = image.crop((x_min, y_min, x_max, y_max))

            # Resize the cropped square to 256x256
            resized_image = cropped_image.resize(resize_size)

            # Save the processed image
            output_path = os.path.join(output_folder, filename)
            resized_image.save(output_path)
            print(f"Processed and saved: {output_path}")

# Call the processing function
intermediate_folder = './data/samsung/samsung/NH3/cover/'  # './data/samsung/samsung/NO2/cover/'
final_folder = './data/samsung/samsung/NH3/middle'      # './samsung/NO2/middle'
crop_and_resize_center_square(intermediate_folder, final_folder)

Processed and saved: ./samsung/NH3/middle\0iOM6_ngochanpham274@gmail.com_2024-11-21 12_52_30_NH3_5ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/middle\1iVDC_ngochanpham274@gmail.com_2024-11-21 12_04_02_NH3_0,25ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/middle\2qR82_ngochanpham274@gmail.com_2024-11-21 12_00_28_NH3_0,09ppm_001_samsung test__.jpg
Processed and saved: ./samsung/NH3/middle\3iW5i_ngochanpham274@gmail.com_2024-11-21 11_48_30_NH3_0,01ppm_003_samsung test_10.8768228_106.6782789.jpg
Processed and saved: ./samsung/NH3/middle\3IwPn_ngochanpham274@gmail.com_2024-11-21 11_59_52_NH3_0,075ppm_003_samsung test__.jpg
Processed and saved: ./samsung/NH3/middle\4fkPP_ngochanpham274@gmail.com_2024-11-21 12_50_09_NH3_4ppm_001_samsung test__.jpg
Processed and saved: ./samsung/NH3/middle\5RgJx_ngochanpham274@gmail.com_2024-11-21 11_55_08_NH3_0,05ppm_002_samsung test_10.8768228_106.6782789.jpg
Processed and saved: ./samsung/NH3/middle\5ufqX_ngochanpham274@gmai